In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
train = pd.read_csv('../train_complete.csv')
train.head()

,ID,State_Name,Year,Age,Incidence,Arithmetic_Mean_mean_Lead_(TSP)_STP,Arithmetic_Mean_min_Lead_(TSP)_STP,Arithmetic_Mean_max_Lead_(TSP)_STP,Arithmetic_Mean_mean_Lead_(TSP)_STP_l2,Arithmetic_Mean_min_Lead_(TSP)_STP_l2,...,X1st_Max_Value_max_Benzene_l8,X1st_Max_Value_max_Ethylbenzene_l2,X1st_Max_Value_max_Ethylbenzene_l5,X1st_Max_Value_max_Ethylbenzene_l8,X1st_Max_Value_max_Toluene_l2,X1st_Max_Value_max_Toluene_l5,X1st_Max_Value_max_Toluene_l8,X1st_Max_Value_max_o-Xylene_l2,X1st_Max_Value_max_o-Xylene_l5,X1st_Max_Value_max_o-Xylene_l8
0,f8312a4,Alabama,1990,65-69,4685.284313,0.664543,0.0,7.96,1.071302,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3effa36,Alabama,1990,70-74,4827.052043,0.664543,0.0,7.96,1.071302,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1e8044b,Alabama,1990,75-79,4377.956914,0.664543,0.0,7.96,1.071302,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,d875d65,Alabama,1990,80-84,3822.732993,0.664543,0.0,7.96,1.071302,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,46e6695,Alabama,1990,85-89,3470.199503,0.664543,0.0,7.96,1.071302,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
train.shape

(4500, 200)

In [9]:
train_dummy = pd.get_dummies(train, columns=['Age'], drop_first=True)
train_dummy.head()

,ID,State_Name,Year,Incidence,Arithmetic_Mean_mean_Lead_(TSP)_STP,Arithmetic_Mean_min_Lead_(TSP)_STP,Arithmetic_Mean_max_Lead_(TSP)_STP,Arithmetic_Mean_mean_Lead_(TSP)_STP_l2,Arithmetic_Mean_min_Lead_(TSP)_STP_l2,Arithmetic_Mean_max_Lead_(TSP)_STP_l2,...,X1st_Max_Value_max_Toluene_l5,X1st_Max_Value_max_Toluene_l8,X1st_Max_Value_max_o-Xylene_l2,X1st_Max_Value_max_o-Xylene_l5,X1st_Max_Value_max_o-Xylene_l8,Age_70-74,Age_75-79,Age_80-84,Age_85-89,Age_90-94
0,f8312a4,Alabama,1990,4685.284313,0.664543,0.0,7.96,1.071302,0.0,28.17,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1,3effa36,Alabama,1990,4827.052043,0.664543,0.0,7.96,1.071302,0.0,28.17,...,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0
2,1e8044b,Alabama,1990,4377.956914,0.664543,0.0,7.96,1.071302,0.0,28.17,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0
3,d875d65,Alabama,1990,3822.732993,0.664543,0.0,7.96,1.071302,0.0,28.17,...,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0
4,46e6695,Alabama,1990,3470.199503,0.664543,0.0,7.96,1.071302,0.0,28.17,...,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0


In [10]:
train_numeric = train_dummy.drop(columns=['ID','State_Name'])

In [11]:
it_imp = IterativeImputer(
    estimator=LinearRegression(),
    max_iter=1000
)

In [12]:
train_imp_missing = it_imp.fit_transform(train_numeric)
train_imp_missing = pd.DataFrame(train_imp_missing, columns=train_numeric.columns)

In [14]:
test = pd.read_csv('../test_complete.csv')
test.head()

,ID,State_Name,Year,Age,Arithmetic_Mean_mean_Lead_(TSP)_STP,Arithmetic_Mean_min_Lead_(TSP)_STP,Arithmetic_Mean_max_Lead_(TSP)_STP,Arithmetic_Mean_mean_Lead_(TSP)_STP_l2,Arithmetic_Mean_min_Lead_(TSP)_STP_l2,Arithmetic_Mean_max_Lead_(TSP)_STP_l2,...,X1st_Max_Value_max_Benzene_l8,X1st_Max_Value_max_Ethylbenzene_l2,X1st_Max_Value_max_Ethylbenzene_l5,X1st_Max_Value_max_Ethylbenzene_l8,X1st_Max_Value_max_Toluene_l2,X1st_Max_Value_max_Toluene_l5,X1st_Max_Value_max_Toluene_l8,X1st_Max_Value_max_o-Xylene_l2,X1st_Max_Value_max_o-Xylene_l5,X1st_Max_Value_max_o-Xylene_l8
0,6c06615,Delaware,1990,65-69,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e0cf76e,Delaware,1990,70-74,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e9f95f0,Delaware,1990,75-79,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5dffe44,Delaware,1990,80-84,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9f9ea9b,Delaware,1990,85-89,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
test_dummy = pd.get_dummies(test, columns=['Age'], drop_first=True)

In [17]:
test_numeric = test_dummy.drop(columns=['ID','State_Name'])

In [18]:
test_imp_missing = it_imp.fit_transform(test_numeric)
test_imp_missing = pd.DataFrame(test_imp_missing, columns=test_numeric.columns)

## Modeling

In [20]:
X = train_imp_missing.drop(columns='Incidence')
y = train_imp_missing['Incidence']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [23]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [24]:
lr.score(X_train,y_train), lr.score(X_test,y_test)

(0.9739042546096413, 0.9744870945171846)

In [25]:
test_preds = lr.predict(test_imp_missing)

In [26]:
test_preds_lr = pd.DataFrame({
    'ID': test['ID'],
    'Incidence': test_preds,
})

test_preds_lr.head()

,ID,Incidence
0,6c06615,-3.600696e+06
1,e0cf76e,-3.599894e+06
2,e9f95f0,-3.599145e+06
3,5dffe44,-3.599270e+06
4,9f9ea9b,-3.600004e+06


In [27]:
test_preds_lr.to_csv('../lr_model_2.csv', index=False)